In [ ]:
'''
boto3 module is the official AWS SDK for Python, 
and it’s one of the most powerful libraries for automating AWS services like EC2, S3, IAM, Lambda, CloudWatch, etc.

boto3 allows Python code to create, configure, and manage AWS resources.
It’s like having the AWS Console in your Python scripts or automation workflows.

pip install boto3
import boto3

| Interface    | Description                             | Example                     |
| ------------ | --------------------------------------- | --------------------------- |
| Client       | Low-level access (direct AWS API calls) |  s3 = boto3.client('s3')   |
| Resource     | High-level, object-oriented API         |  s3 = boto3.resource('s3') |

'''

'''
Authentication

boto3 automatically finds your AWS credentials from:
aws configure (CLI)
Environment variables
(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
IAM roles (EC2, Lambda, etc.)
Shared credentials file (~/.aws/credentials)

Example:
aws configure
Creates ~/.aws/credentials with your keys.

'''

In [ ]:
# List S3 Buckets

import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

for bucket in response['Buckets']:
    print(bucket['Name'])

# my-data-bucket
# project-logs


In [ ]:
# Upload / Download Files (S3)

s3 = boto3.client('s3')

# Upload
s3.upload_file("local.txt", "my-bucket", "backup/local.txt")

# Download
s3.download_file("my-bucket", "backup/local.txt", "restored.txt")

# Common for backups, logs, data pipelines.

In [ ]:
# Using S3 Resource (Object-Oriented)

s3 = boto3.resource('s3')

bucket = s3.Bucket('my-bucket')
for obj in bucket.objects.all():
    print(obj.key)


# data/file1.txt
# data/file2.csv


In [ ]:
# EC2 Example – List Instances

ec2 = boto3.client('ec2')

for instance in ec2.describe_instances()['Reservations']:
    for i in instance['Instances']:
        print(i['InstanceId'], i['State']['Name'])

# i-12345abc9f2a1234 running
# i-12345abc9f2a1234 stopped


In [ ]:
# Start or Stop an EC2 Instance

ec2 = boto3.client('ec2')

# Start instance
ec2.start_instances(InstanceIds=['i-12345abc9f2a1234'])

# Stop instance
ec2.stop_instances(InstanceIds=['i-12345abc9f2a1234'])


In [ ]:
# IAM Example – List Users
iam = boto3.client('iam')

for user in iam.list_users()['Users']:
    print(user['UserName'])

# admin
# devops-engineer


In [ ]:
# CloudWatch Example – List Metrics

cloudwatch = boto3.client('cloudwatch')

metrics = cloudwatch.list_metrics(Namespace='AWS/EC2')
for m in metrics['Metrics'][:5]:
    print(m)

# {'Namespace': 'AWS/EC2', 'MetricName': 'CPUUtilization', 'Dimensions': [{'Name': 'InstanceId', 'Value': 'i-0123...'}]}


In [ ]:
# Lambda Example – Invoke Function
lambda_client = boto3.client('lambda')

response = lambda_client.invoke(
    FunctionName='myLambdaFunction',
    InvocationType='RequestResponse',
    Payload=b'{"key": "value"}'
)

print(response['Payload'].read().decode())


In [ ]:
# Create a New S3 Bucket
s3 = boto3.client('s3')
s3.create_bucket(Bucket='new-bucket-example-2025')


In [ ]:
# Working with Regions
ec2 = boto3.client('ec2', region_name='us-east-1')


In [ ]:
# Error Handling
# Always use botocore.exceptions for clean error handling.

from botocore.exceptions import NoCredentialsError, ClientError

try:
    s3.upload_file("local.txt", "my-bucket", "data/local.txt")
except FileNotFoundError:
    print("File not found!")
except NoCredentialsError:
    print("Credentials not available!")
except ClientError as e:
    print("AWS Error:", e)


In [ ]:
# Pagination Example (Large Results)

ec2 = boto3.client('ec2')
paginator = ec2.get_paginator('describe_instances')

for page in paginator.paginate():
    for reservation in page['Reservations']:
        for instance in reservation['Instances']:
            print(instance['InstanceId'])


In [ ]:
'''
| Category            | Example                                     | Description               |
| ------------------- | ------------------------------------------- | ------------------------- |
| **Setup**           | `boto3.client('s3')`                        | Create AWS service client |
| **List resources**  | `.list_buckets()` / `.describe_instances()` | Retrieve AWS resources    |
| **Modify**          | `.start_instances()`, `.create_bucket()`    | Manage resources          |
| **Upload/Download** | `upload_file()` / `download_file()`         | File operations           |
| **Error Handling**  | `try...except ClientError`                  | Handle AWS errors         |
| **Authentication**  | `aws configure`                             | Secure access             |

'''

In [ ]:
# Real-World DevOps Example — Backup to S3
import boto3
import shutil
from datetime import datetime

s3 = boto3.client("s3")

# Create local zip backup
backup_name = f"backup_{datetime.now().strftime('%Y%m%d_%H%M')}.zip"
shutil.make_archive("logs_backup", "zip", "logs")

# Upload to S3
s3.upload_file("logs_backup.zip", "my-backup-bucket", backup_name)
print("Backup uploaded to S3!")
